In [3]:
# | default_exp configs.manager

In [9]:
# | export
import os
import toml
from pathlib import Path
from typing import Dict, Any, Optional
from dataclasses import dataclass, field

In [10]:
# | export
@dataclass
class ModelConfig:
    """Model configuration"""
    name: str = "qwen3:8b"
    url: str = "http://localhost:11434/v1"
    api_key: str = "ollama"
    temperature: float = 0.7
    max_tokens: Optional[int] = None
    timeout: int = 60


@dataclass
class SettingsConfig:
    """General settings configuration"""
    auto_approve: bool = False
    stream: bool = True
    debug: bool = False
    log_level: str = "INFO"
    max_history: int = 100


@dataclass
class ToolsConfig:
    """Tools configuration"""
    default_tools: list = field(default_factory=lambda: [
        "fs_read", "fs_write", "execute_bash", "code_interpreter"
    ])
    dangerous_tools: list = field(default_factory=lambda: [
        "execute_bash", "fs_write"
    ])
    require_approval: list = field(default_factory=lambda: [
        "execute_bash", "fs_write", "code_interpreter"
    ])


@dataclass
class ReasoningConfig:
    """Reasoning and thinking configuration"""
    show_thinking: bool = True
    thinking_color: str = "pink"
    save_thinking: bool = False
    retry_count: int = 2
    max_reasoning_steps: int = 10

@dataclass
class MCPConfig:
    enabled: bool = False
    servers: list = field(default_factory=lambda: [
    ])
    
@dataclass
class AgenticConfig:
    """Complete agentic configuration"""
    model: ModelConfig = field(default_factory=ModelConfig)
    settings: SettingsConfig = field(default_factory=SettingsConfig)
    tools: ToolsConfig = field(default_factory=ToolsConfig)
    reasoning: ReasoningConfig = field(default_factory=ReasoningConfig)
    mcp: MCPConfig = field(default_factory=MCPConfig)


class ConfigManager:
    """configuration manager"""
    
    def __init__(self, config_path: Optional[str] = None):
        self.config_path = config_path or self._get_default_config_path()
        self.config = self._load_config()
    
    def _get_default_config_path(self) -> str:
        """Get default config file path"""
        current_dir = Path(__file__).parent
        return str(current_dir / "config.toml")
    
    def _load_config(self) -> AgenticConfig:
        """Load configuration from file"""
        try:
            if os.path.exists(self.config_path):
                with open(self.config_path, 'r') as f:
                    config_data = toml.load(f)
                
                return AgenticConfig(
                    model=ModelConfig(**config_data.get('model', {})),
                    settings=SettingsConfig(**config_data.get('settings', {})),
                    tools=ToolsConfig(**config_data.get('tools', {})),
                    reasoning=ReasoningConfig(**config_data.get('reasoning', {})),
                    mcp=MCPConfig(**config_data.get('mcp', {}))
                )
            else:
                # Create default config file
                default_config = AgenticConfig()
                self._save_config(default_config)
                return default_config
                
        except Exception as e:
            print(f"Warning: Failed to load config from {self.config_path}: {e}")
            return AgenticConfig()
    
    def _save_config(self, config: AgenticConfig):
        """Save configuration to file"""
        try:
            config_data = {
                'model': {
                    'name': config.model.name,
                    'url': config.model.url,
                    'api_key': config.model.api_key,
                    'temperature': config.model.temperature,
                    'max_tokens': config.model.max_tokens,
                    'timeout': config.model.timeout
                },
                'settings': {
                    'auto_approve': config.settings.auto_approve,
                    'stream': config.settings.stream,
                    'debug': config.settings.debug,
                    'log_level': config.settings.log_level,
                    'max_history': config.settings.max_history
                },
                'tools': {
                    'default_tools': config.tools.default_tools,
                    'dangerous_tools': config.tools.dangerous_tools,
                    'require_approval': config.tools.require_approval
                },
                'reasoning': {
                    'show_thinking': config.reasoning.show_thinking,
                    'thinking_color': config.reasoning.thinking_color,
                    'save_thinking': config.reasoning.save_thinking,
                    'retry_count': config.reasoning.retry_count,
                    'max_reasoning_steps': config.reasoning.max_reasoning_steps
                }
            }
            
            # Ensure directory exists
            os.makedirs(os.path.dirname(self.config_path), exist_ok=True)
            
            with open(self.config_path, 'w') as f:
                toml.dump(config_data, f)
                
        except Exception as e:
            print(f"Warning: Failed to save config to {self.config_path}: {e}")
    
    def get_model_config(self) -> Dict[str, Any]:
        """Get model configuration as dict"""
        return {
            'name': self.config.model.name,
            'url': self.config.model.url,
            'api_key': self.config.model.api_key,
            'temperature': self.config.model.temperature,
            'max_tokens': self.config.model.max_tokens,
            'timeout': self.config.model.timeout
        }
    
    def get_settings_config(self) -> Dict[str, Any]:
        """Get settings configuration as dict"""
        return {
            'auto_approve': self.config.settings.auto_approve,
            'stream': self.config.settings.stream,
            'debug': self.config.settings.debug,
            'log_level': self.config.settings.log_level,
            'max_history': self.config.settings.max_history
        }
    
    def get_tools_config(self) -> Dict[str, Any]:
        """Get tools configuration as dict"""
        return {
            'default_tools': self.config.tools.default_tools,
            'dangerous_tools': self.config.tools.dangerous_tools,
            'require_approval': self.config.tools.require_approval
        }
    
    def get_reasoning_config(self) -> Dict[str, Any]:
        """Get reasoning configuration as dict"""
        return {
            'show_thinking': self.config.reasoning.show_thinking,
            'thinking_color': self.config.reasoning.thinking_color,
            'save_thinking': self.config.reasoning.save_thinking,
            'retry_count': self.config.reasoning.retry_count,
            'max_reasoning_steps': self.config.reasoning.max_reasoning_steps
        }
    
    def update_config(self, section: str, updates: Dict[str, Any]):
        """Update configuration section"""
        if section == 'model':
            for key, value in updates.items():
                if hasattr(self.config.model, key):
                    setattr(self.config.model, key, value)
        elif section == 'settings':
            for key, value in updates.items():
                if hasattr(self.config.settings, key):
                    setattr(self.config.settings, key, value)
        elif section == 'tools':
            for key, value in updates.items():
                if hasattr(self.config.tools, key):
                    setattr(self.config.tools, key, value)
        elif section == 'reasoning':
            for key, value in updates.items():
                if hasattr(self.config.reasoning, key):
                    setattr(self.config.reasoning, key, value)
        elif section == 'mcp':
            for key, value in updates.items():
                if hasattr(self.config.mcp, key):
                    setattr(self.config.mcp, key, value)
        
        # Save updated config
        self._save_config(self.config)
    
    def reset_to_defaults(self):
        """Reset configuration to defaults"""
        self.config = AgenticConfig()
        self._save_config(self.config)


# Global config manager instance
_config_manager = None


def get_config_manager() -> ConfigManager:
    """Get global config manager instance"""
    global _config_manager
    if _config_manager is None:
        _config_manager = ConfigManager()
    return _config_manager


In [3]:
from agentic.configs.manager import get_config_manager
get_config_manager().config.mcp

MCPConfig(enabled=True, servers=[{'name': 'filesystem', 'command': ['npx', '@modelcontextprotocol/server-filesystem'], 'args': ['/home/pranav-pc/projects/applied-GenAI-lab'], 'description': 'File operations - read, write, list, search files'}, {'name': 'thinking', 'command': ['npx', '@modelcontextprotocol/server-sequential-thinking'], 'args': [], 'description': 'Structured problem solving and step-by-step thinking'}, {'name': 'context7', 'command': ['npx', '@upstash/context7-mcp'], 'args': [], 'description': 'Library documentation and code examples'}, {'name': 'browser', 'command': ['npx', 'puppeteer-mcp-server'], 'args': [], 'description': 'Web browser automation and scraping'}])